## Nutritional information from openfoodfacts

In [1]:
import requests, six
import lxml.html as lh
from itertools import cycle, islice
from matplotlib import colors
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
%matplotlib inline
import openfoodfacts

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("energy_tracker.db")
sugg_df = pd.read_sql_query("SELECT * from suggested_energy_intake", con)

# Verify that result of SQL query is stored in the dataframe
display(sugg_df.head(1))

sugg_df["Daily requirements"]=sugg_df["Daily requirements"].astype(float)
_ = sugg_df.rename({"Daily requirements": "amount"}, inplace=True, axis=1)

con.close()

,Nutrient,Daily requirements,Units
0,Calorie,1500.0,kcal


In [4]:
barcodes = pd.read_csv("Scan session.csv", header=None).T.to_numpy().tolist()[0]
np.savetxt("food_barcodes.csv", barcodes + [5411188512271, 8718403014549], delimiter=",", fmt='%s')
barcodes = pd.read_csv("food_barcodes.csv", header=None).T.to_numpy().tolist()[0]

In [5]:
np.array(barcodes)

array([8714779001053, 8718452427512, 8711852200181, 5411188111252,
       7394376616419, 8718452265732, 8718452459698, 5202390019319,
       8714779003187, 8718452451043, 8719324157025, 8710624281052,
       8710624280499, 8710624280437, 5060072970019, 8711299021738,
       8711299021776, 8711299021721, 8711299021714,      50818853,
       8718452334827, 8718452261697, 8713965500189, 8718452375783,
       8710398517388, 8690661080330, 8719153025236, 8718452396535,
       8718452469352, 8719189108200, 8715700131207, 8718452469352,
       8718452375783, 8713965500189, 8690661080330, 8718452459698,
       8718452459698, 5411188111078, 8719153025236, 8710398517388,
       8718692781863, 8710861912900, 5411188512271, 8718403014549])

In [6]:
all_foods_df = pd.DataFrame()
failed_barcodes = list()
for barcode_int in barcodes:
    barcode=str(barcode_int)
    try:
        search_result = openfoodfacts.products.get_product(barcode)
        food_df = pd.concat([pd.DataFrame({"product_name":search_result["product"]["product_name"],
                                          "barcode":barcode}, index=[0]), 
                         pd.DataFrame(search_result["product"]["nutriments"], index=[0])], axis=1)
        all_foods_df = pd.concat([all_foods_df, food_df])
    except:
        print(f"Barcode == {barcode} not available in the database")
        failed_barcodes.append(barcode_int)
_ = all_foods_df.reset_index(inplace=True, drop=True)

Barcode == 8718452396535 not available in the database
Barcode == 8719189108200 not available in the database


In [7]:
all_foods_df.filter(regex="100g").drop("energy_100g", axis=1).head(1)

,carbohydrates_100g,energy-kcal_100g,fat_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g,proteins_100g,salt_100g,saturated-fat_100g,sodium_100g,sugars_100g,fiber_100g,carbon-footprint-from-known-ingredients_100g,nova-group_100g,energy-kj_100g,vitamin-b1_100g,vitamin-k_100g,biotin_100g,magnesium_100g,phosphorus_100g,vitamin-b9_100g,vitamin-e_100g,vitamin-pp_100g,calcium_100g,vitamin-b12_100g,vitamin-b2_100g,vitamin-d_100g
0,24.8,232,9.7,0.0,3.0,8.4,1.2,0.9,0.48,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
all_foods_df.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

In [18]:
# dir(openfoodfacts.products)
# openfoodfacts.products.search("apple")

In [10]:
all_foods_df

,product_name,barcode,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy_100g,energy_unit,energy_value,fat,fat_100g,fat_unit,fat_value,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr,nutrition-score-fr_100g,proteins,proteins_100g,proteins_unit,proteins_value,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium,sodium_100g,sodium_unit,sodium_value,sugars,sugars_100g,sugars_unit,sugars_value,carbohydrates_serving,energy-kcal_serving,energy_serving,fat_serving,fiber,fiber_100g,fiber_serving,fiber_unit,fiber_value,proteins_serving,saturated-fat_serving,sugars_serving,carbon-footprint-from-known-ingredients_100g,nova-group,nova-group_100g,nova-group_serving,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,energy-kj,energy-kj_100g,energy-kj_serving,energy-kj_unit,energy-kj_value,salt_serving,sodium_serving,vitamin-b1,vitamin-b1_100g,vitamin-b1_serving,vitamin-b1_unit,vitamin-b1_value,vitamin-k,vitamin-k_100g,vitamin-k_unit,vitamin-k_value,salt_modifier,sodium_modifier,biotin,biotin_100g,biotin_unit,biotin_value,magnesium,magnesium_100g,magnesium_unit,magnesium_value,phosphorus,phosphorus_100g,phosphorus_unit,phosphorus_value,vitamin-b9,vitamin-b9_100g,vitamin-b9_unit,vitamin-b9_value,vitamin-e,vitamin-e_100g,vitamin-e_unit,vitamin-e_value,vitamin-pp,vitamin-pp_100g,vitamin-pp_unit,vitamin-pp_value,calcium,calcium_100g,calcium_label,calcium_unit,calcium_value,vitamin-b12,vitamin-b12_100g,vitamin-b12_label,vitamin-b12_unit,vitamin-b12_value,vitamin-b2,vitamin-b2_100g,vitamin-b2_label,vitamin-b2_unit,vitamin-b2_value,vitamin-d,vitamin-d_100g,vitamin-d_label,vitamin-d_unit,vitamin-d_value
0,Falafel,8714779001053,24.80,24.80,g,24.80,971,232,232,kcal,232,971,kcal,232,9.70,9.70,g,9.70,0.0,3.0,3.0,8.40,8.40,g,8.40,1.2000,1.2000,g,1.2000,0.900,0.900,g,0.900,0.4800,0.4800,g,0.4800,6.60,6.60,g,6.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jumbo zoete dip van paprika walnoten en couscous,8718452427512,15.00,15.00,g,15.00,866,207,207,kcal,207,866,kcal,207,14.40,14.40,g,14.40,NaN,NaN,NaN,3.20,3.20,g,3.20,NaN,NaN,NaN,NaN,1.200,1.200,g,1.200,NaN,NaN,NaN,NaN,7.50,7.50,g,7.50,15.0,207.0,866.0,14.4,2.3,2.3,2.3,g,2.3,3.2,1.2,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hummus pikant,8711852200181,12.40,12.40,g,12.40,1071,256,256,kcal,256,1071,kcal,256,19.10,19.10,g,19.10,NaN,NaN,NaN,6.20,6.20,g,6.20,1.3400,1.3400,g,1.3400,1.900,1.900,g,1.900,0.5360,0.5360,g,0.5360,1.40,1.40,g,1.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Soja lait,5411188111252,2.50,2.50,g,2.50,163,39,39,kcal,39,163,kcal,39,1.80,1.80,g,1.80,NaN,0.0,0.0,0.80,0.80,g,0.80,0.0800,0.0800,g,0.0800,0.300,0.300,g,0.300,0.0320,0.0320,g,0.0320,2.50,2.50,g,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Haferdrink 

In [9]:
stop!

SyntaxError: invalid syntax (<ipython-input-9-8c726f94684c>, line 1)

In [ ]:
_ = plt.figure(figsize=(6, 5))
for colname in all_foods_df.filter(regex="100g").drop("energy_100g", axis=1).dropna(axis=1).columns.tolist():
    _ = plt.barh(all_foods_df["product_name"], all_foods_df[colname])
    _ = plt.axvline(all_foods_df[colname].median(), color="red", linestyle="--")
    _ = plt.text(x=all_foods_df[colname].median()+all_foods_df[colname].describe()["max"]/14,
                 y=0,
                 s=f'Median amount == {all_foods_df[colname].median().round(2)}',color="red", fontsize=12)
    _ = plt.title(f"{colname} amounts")
    _ = plt.show()

In [ ]:
sugg_df.T

In [ ]:
def unit_conversion(amount, unit):
    if unit == "mg":
        return amount / 1000
    elif unit == "μg":
        return amount / 100000
    else:
        return amount

In [ ]:
sugg_df["converted_amount"] = sugg_df.apply(lambda row: unit_conversion(row['amount'], row['Units']), axis=1)

In [ ]:
plt.figure(figsize=(6,5))
for i, j in [0, 0.1], [0.1, 10], [10, 300]:
    _ = sns.barplot(data=sugg_df[sugg_df["converted_amount"].between(i, j)], x="converted_amount", y="Nutrient")
    _ = plt.show()

In [ ]:
# (markers, stemlines, baseline) = plt.stem(all_foods_df.index, all_foods_df["energy-kcal"])
# _ = plt.setp(markers, marker='D', markersize=6, markeredgecolor="orange", markeredgewidth=2)
# _ = plt.axhline(all_foods_df["energy-kcal"].median(), color="red", linestyle="--")
# _ = plt.ylabel("Kcal")

In [ ]:
stop!

### upload csv to sql table

In [ ]:
# # Read sqlite query results into a pandas DataFrame
# conn = sqlite3.connect("energy_tracker.db")
# df = pd.read_csv("Suggested_nutritional_intake.csv",sep=';',quotechar='\'',encoding='utf8') # Replace Excel_file_name with your excel sheet name
# df.to_sql('suggested_energy_intake_v2',con=con,index=False,if_exists='append') # Replace Table_name with your sql table name